In [63]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2,ifft2,fftfreq
import sympy as sy
from sympy import sin, exp, cos
import pandas as pd
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings("ignore")
from matplotlib.animation import FuncAnimation,  PillowWriter

### Transient Biharmonic Solver

First of all, I will use a transient biharmonic solver to solve the biharmonic heat equation. It is a great warm up exercise to get used to solving PDEs numerically in the frequency domain.

I am going to use the $\theta$-method as the numerical differencial equation solver;

$
U^{n+1} = U^n + \tau \left( \theta F(t_{n+1}, U^{n+1}) + (1 - \theta) F(t_n, U^n) \right)$

$
\partial_t U = F(t, U), \qquad U(t_0) = U_0
$


to solve the equation:

$
\partial_t u + \kappa \Delta^2 u = g
$

The equations above gives:

$F(t, U) = g - \kappa \Delta^2 u$

In frequency space we have that:

$\tilde{k} = 2 \pi (k_x,k_y)$
  
$F(t, U) = \hat{g} -\kappa |\tilde{k}|^4 \hat{u}$

This gives:

$F(t_n, U^n) = \hat{g} -\kappa |\tilde{k}|^4 \hat{u}$  


$F(t_{n+1}, U^{n+1}) = \hat{g}^{n+1} -\kappa |\tilde{k}|^4 \hat{u}^{n+1}$  

We input this in the $\theta$-method and get:

$\hat{u}^{n+1} = \hat{u}^{n} + \tau(\theta(\hat{g}^{n+1} -\kappa |\tilde{k}|^4 \hat{u}^{n+1}) + (1-\theta)(\hat{g} -\kappa |\tilde{k}|^4 \hat{u}))$

Which can be written as:  

$\hat{u}^{n+1} = \frac{\hat{u}^{n}(1-\tau(1-\theta)\kappa|\tilde{k}|^4) + \tau(\theta \hat{g}^{n+1} + (1-\theta)\hat{g})}{1+\tau\theta\kappa\|\tilde{k}|^4}$

This is the iterative method used in the transient biharmonic solver.

In [64]:
def transient_biharmonic_solver(*,kappa,X,Y,U0,t0,T,Nt,theta,g = None):
    x, y = X[0,:], Y[:,0]
    Nx, Ny = len(x), len(y)
    dx, dy = x[1] - x[0], y[1] - y[0]


    #Calculating frequencies in frequency space
    kx = fftfreq(Nx, d=dx/(2*np.pi))
    ky = fftfreq(Ny, d=dy/(2*np.pi))
    KX, KY = np.meshgrid(kx, ky, indexing='ij')

    #Biharmonic operator in frequency space
    K4 = (KX**2+KY**2)**2

    #Time step
    t = t0
    dt = (T-t0)/Nt
    U_hat = fft2(U0)

    #Returing initial conditions
    yield (U_hat, t)
    
    while t < T-dt/2:
        if g is not None:
            G_hat = fft2(g(X,Y,t))
            G_hat_next = fft2(g(X,Y,t+dt))
        else:
            G_hat = 0
            G_hat_next = 0

        U_hat = (U_hat*(1-dt*(1-theta)*kappa*K4) + (dt*((theta*G_hat_next)+((1-theta)*G_hat))))/(1+dt*theta*kappa*K4)
            
        t = t + dt
        yield U_hat, t

In [65]:
#Calculation of the right hand side of the equation using sympy:
def manufacture_solution_transistent(u_string,kappa):

    x, y, t = sy.symbols('x y t')
    u_symbolic = eval(u_string)

    laplace = lambda u: sy.diff(u, x, 2) + sy.diff(u, y, 2)
    
    firstLaplace = laplace(u_symbolic)
    biharmonic = laplace(firstLaplace)
    g_symbolic = kappa*biharmonic + sy.diff(u_symbolic,t,1) 

    u = sy.lambdify((x, y, t), u_symbolic, modules='numpy')
    g = sy.lambdify((x, y, t), g_symbolic, modules='numpy')

    return u,g

In [66]:
#Parameters
Lx,Ly = np.pi,np.pi
Nx,Ny = 20,20
x = np.linspace(-Lx,Lx,Nx,endpoint=False)
y = np.linspace(-Ly,Ly,Ny,endpoint=False)
X,Y = np.meshgrid(x,y,sparse=True)

kappa = 1
T,t0 = 1,0

#For lambda = 4, we obtain g(x,y,t) = 0. 
lamb = 4
Nt_list = [10,20,40,80,160,320,640]
g = None

#Analitic solution
u_string_ex = 'sin(x)*cos(y)*exp(-lamb*kappa*t)'
u_ex,g_exact = manufacture_solution_transistent(u_string_ex,kappa)

u0 = u_ex
U0 = u0(X,Y,0)

In [67]:
#Function that calculates errors and the experimental order of convergence (EOC) for each time step.
def compute_eoc_transient(*,
                          kappa, u_ex, U0, g,
                          X, Y, t0,theta, T, Nt_list):
    errs_Nt = [] 
    for Nt in Nt_list:
        U_list = transient_biharmonic_solver(kappa=kappa,X=X,Y=Y,U0=U0,
                                        t0=t0,T=T,Nt=Nt,theta=theta,g = g)
        errs_t = []
        for U, t in U_list:
            U = ifft2(U).real
            U_ex = u_ex(X,Y,t)
            U_err = U - U_ex
            errs_t.append(np.linalg.norm(U_err, np.inf))
        errs_Nt.append(np.array(np.linalg.norm(errs_t, np.inf)))
        
    Nt_list = np.array(Nt_list)
    errs_Nt = np.array(errs_Nt)
    eocs = np.log(errs_Nt[1:]/errs_Nt[:-1])/np.log(Nt_list[:-1]/Nt_list[1:])
    eocs = np.insert(eocs, 0, np.inf)
    return errs_Nt, eocs

In [68]:
u0 = u_ex
U0 = u0(X,Y,0)

errs1, eocs1 = compute_eoc_transient(kappa=kappa, 
                                   u_ex=u0, U0=U0, g=g,
                                   X=X, Y=Y, t0=t0,theta=1, T=T, Nt_list=Nt_list)
errs2, eocs2 = compute_eoc_transient(kappa=kappa, 
                                   u_ex=u0, U0=U0, g=g,
                                   X=X, Y=Y, t0=t0,theta=0.5, T=T, Nt_list=Nt_list)

errs3, eocs3 = compute_eoc_transient(kappa=kappa, 
                                   u_ex=u0, U0=U0, g=g,
                                   X=X, Y=Y, t0=t0,theta=0, T=T, Nt_list=Nt_list)    

display(Markdown(r"### 📊 Convergence table for $\theta = 1$"))
table = pd.DataFrame({'Nt': Nt_list, 'error': errs1, 'EOC': eocs1})
display(table)
display(Markdown(r"### 📊 Convergence table for $\theta = 0.5$"))
table = pd.DataFrame({'Nt': Nt_list, 'error': errs2, 'EOC': eocs2})
display(table)
display(Markdown(r"### 📊 Convergence table for $\theta = 0$"))
table = pd.DataFrame({'Nt': Nt_list, 'error': errs3, 'EOC': eocs3})
display(table)

### 📊 Convergence table for $\theta = 1$

,Nt,error,EOC
0,10,0.798529,inf
1,20,0.429312,0.895320
2,40,0.223050,0.944656
3,80,0.113774,0.971194
4,160,0.057470,0.985292
5,320,0.028883,0.992567
6,640,0.014479,0.996263


### 📊 Convergence table for $\theta = 0.5$

,Nt,error,EOC
0,10,0.061848,inf
1,20,0.015552,1.991623
2,40,0.003875,2.004710
3,80,0.000968,2.001173
4,160,0.000242,2.000293
5,320,0.000060,2.000073
6,640,0.000015,2.000018


### 📊 Convergence table for $\theta = 0$

,Nt,error,EOC
0,10,1.752704e+19,inf
1,20,9.810651e+48,-98.820681
2,40,8.019770e+102,-179.093329
3,80,5.868343e+198,-318.454490
4,160,NaN,NaN
5,320,NaN,NaN
6,640,NaN,NaN


With $\theta = 0.5$ we obtain the largest experimental order of convergence. This is expected, because $\theta = 0.5$ yields the Crank-Nicholson method, which is of order 2.

### IMEX-solver

I will now implement an implicit-explicit method. This is a method that solves some parts of an equation implicit and some parts explicit. I will still use the $\theta$-method as my differencial equation solver.

We have the Cahn-Hilliard equation:  

$
\partial_t u - \nabla \cdot \left( M \nabla(f(u)) - \kappa \Delta u \right) = g
$

$
\partial_t u = g + M\Delta(f(u)) + M\kappa\Delta^2u
$

$\Delta f(u)$ is defined as:
  
$\Delta f(u) = \Delta(u^3-u) = \alpha\Delta u + \Delta u^3 - (1+\alpha)\Delta u $

Vi får da:  


$
\partial_t u = g + M(\alpha\Delta u + \Delta u^3 - (1+\alpha)\Delta u) + M\kappa\Delta^2u
$


Now we can partition into an explicit part and an implicit part:

Explicit part:  

$F(t_n,U^n) = M(\Delta (u^n)^3 - (1+\alpha)\Delta u^n)$

Implicit part:

$F(t_n + \tau, U^{n+1}) = g^{n+1} + M \alpha \Delta u^{n+1} - M \kappa \Delta^2 u^{n+1}$  

Combining them into one expression:  

$u^{n+1} = u^n + \tau M(\Delta (u^n)^3 - (1+\alpha)\Delta u^n) + \tau (g^{n+1} + M \alpha \Delta u^{n+1} - M \kappa \Delta^2 u^{n+1})$  


Fourier-transforming the expression:

$
\hat{u}^{n+1} = \hat{u}^n - \tau M |\tilde{K}|^2 \left( \widehat{(u^n)^3 - (1+\alpha) \hat{u}^n} \right) + \tau \left( \hat{g}^{n+1} - M \alpha |\tilde{K}|^2 \hat{u}^{n+1} - M \kappa |\tilde{K}|^4 \hat{u}^{n+1} \right)
$  

$
\hat{u}^{n+1} = \frac{\hat{u}^n - \tau M |\tilde{K}|^2 \left( \widehat{(u^n)^3 - (1+\alpha) \hat{u}^n} \right) + \tau \hat{g}^{n+1}}{1 + \tau M \alpha |\tilde{K}|^2 + \tau M \kappa |\tilde{K}|^4}
$  

The method calculates $(u^n)^3 - (1+\alpha) u^n$ in the spatial domain, and fourier-transforms it using the DFT later. This is practical, as we are not needed to do the tough calulation of a convolution of a non-linear term.

In [69]:
def cahn_hilliard_backwards_euler(*,kappa,X,Y,U0,t0,T,Nt,alpha,g = None,tau = None):
    x, y = X[0,:], Y[:,0]
    Nx, Ny = len(x), len(y)
    dx, dy = x[1] - x[0], y[1] - y[0]


    #Calculating the frequencies in the frequency domain
    kx = fftfreq(Nx, d=dx/(2*np.pi))
    ky = fftfreq(Ny, d=dy/(2*np.pi))
    KX, KY = np.meshgrid(kx, ky, indexing='ij')

    #Biharmonic operator in the frequency domain
    K4 = (KX**2+KY**2)**2

    #Laplacian operator in the frequency domain
    K2 = KX**2+KY**2

    #Time-step
    t = t0
    dt = (T-t0)/Nt
    U_hat = fft2(U0)

    #Allows us to manually set the time step for later
    if tau is not None:
        dt = tau

    #Return initial conditions
    yield (U_hat, t)
    
    
    while t < T-dt/2:
        if g is not None:
            G_hat = fft2(g(X,Y,t))
        else:
            G_hat = 0

        #Transforming the U-vector to the spatial domain
        U_real_space =ifft2(U_hat).real

        #Fourier transforming the non-linear term
        nonlinear = U_real_space**3 - (1+alpha)*U_real_space
        nonlinearFourier = fft2(nonlinear)
        
        #Updating the U-vector
        teller = U_hat + dt*(G_hat-M*K2*nonlinearFourier)
        nevner = 1 + dt*(M*alpha*K2 + kappa*K4)
        U_hat = teller/nevner

        #Setting the mean value
        U_hat[0, 0] = np.mean(ifft2(U_hat)).real * Nx * Ny

        t = t + dt
        yield U_hat, t

In [70]:
#Manufacture analytic solutions
def manufacture_solution_cahnHilliard(u_string,kappa,M=1):

    x, y, t = sy.symbols('x y t')
    u_symbolic = eval(u_string)

    laplace = lambda u: sy.diff(u, x, 2) + sy.diff(u, y, 2)

    uDerivative = sy.diff(u_symbolic, t)
    uBiharmonic = kappa*laplace(laplace(u_symbolic))
    uLaplace = M*laplace(u_symbolic**3-u_symbolic)
    g_symbolic = uDerivative + uBiharmonic - uLaplace

    u = sy.lambdify((x, y, t), u_symbolic, modules='numpy')
    g = sy.lambdify((x, y, t), g_symbolic, modules='numpy')

    return u,g

In [71]:
#Function calculating errors and EOC for different time steps
def compute_eoc_cahnHilliard(*,
                          kappa, u_ex, U0, g,
                          X, Y, t0,alpha, T, Nt_list):
    errs_Nt = [] 
    for Nt in Nt_list:
        U_list = cahn_hilliard_backwards_euler(kappa=kappa,X=X,Y=Y,U0=U0,
                                        t0=t0,T=T,Nt=Nt,alpha = alpha,g = g)
        errs_t = []
        for U, t in U_list:
            U = ifft2(U).real
            U_ex = u_ex(X,Y,t)
            U_err = U - U_ex
            errs_t.append(np.linalg.norm(U_err, np.inf))
        errs_Nt.append(np.array(np.linalg.norm(errs_t, np.inf)))
        
    Nt_list = np.array(Nt_list)
    errs_Nt = np.array(errs_Nt)
    eocs = np.log(errs_Nt[1:]/errs_Nt[:-1])/np.log(Nt_list[:-1]/Nt_list[1:])
    eocs = np.insert(eocs, 0, np.inf)
    return errs_Nt, eocs

In [72]:
#Analytic solutions
u_string_ex = 'sin(x)*cos(y)*exp(-4*kappa*t)'
u_ex0,g_exact0 = manufacture_solution_cahnHilliard(u_string_ex,kappa=1)
u_ex1,g_exact1 = manufacture_solution_cahnHilliard(u_string_ex,kappa=0.01)

#Preparations before computing
Lx,Ly = 16*np.pi,16*np.pi
Nx,Ny =64,64
x = np.linspace(0,Lx,Nx,endpoint=False)
y = np.linspace(0,Ly,Ny,endpoint=False)
X,Y = np.meshgrid(x,y,sparse=True)

#Used parameters
kappa = 1
T,t0 = 1,0
alpha2 = 1.5
alpha1 = 0.5
alpha3 = 3
M= 1

Nt_list = [100,200,400,800,1600,3200]

#kappa = 1
gk0 = g_exact0
uk0 = u_ex0
Uk0 = uk0(X,Y,0)

#kappa = 0.01
gk1 = g_exact1
uk1 = u_ex1
Uk1 = uk1(X,Y,0)

                    
errs1, eocs1 = compute_eoc_cahnHilliard(
                          kappa=1, u_ex=uk0, U0=Uk0, g=gk0,
                          X=X, Y=Y, t0=t0,alpha=alpha1, T=T, Nt_list=Nt_list)
errs12, eocs12 = compute_eoc_cahnHilliard(
                          kappa=1, u_ex=uk0, U0=Uk0, g=gk0,
                          X=X, Y=Y, t0=t0,alpha=alpha2, T=T, Nt_list=Nt_list)
errs13, eocs13 = compute_eoc_cahnHilliard(
                          kappa=1, u_ex=uk0, U0=Uk0, g=gk0,
                          X=X, Y=Y, t0=t0,alpha=alpha3, T=T, Nt_list=Nt_list)

errs2, eocs2 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=uk1, U0=Uk1, g=gk1,
                          X=X, Y=Y, t0=t0,alpha=alpha1, T=T, Nt_list=Nt_list)

errs3, eocs3 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=uk1, U0=Uk1, g=gk1,
                          X=X, Y=Y, t0=t0,alpha=alpha2, T=T, Nt_list=Nt_list)

errs4, eocs4 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=uk1, U0=Uk1, g=gk1,
                          X=X, Y=Y, t0=t0,alpha=alpha3, T=T, Nt_list=Nt_list)


display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 0.5$" ))
table1 = pd.DataFrame({'Nt': Nt_list, 'error': errs1, 'EOC': eocs1})
display(table1)
display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 0.01$, $\alpha = 0.5$"))
table2 = pd.DataFrame({'Nt': Nt_list, 'error': errs2, 'EOC': eocs2})
display(table2)

display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 1.5$" ))
table1 = pd.DataFrame({'Nt': Nt_list, 'error': errs12, 'EOC': eocs12})
display(table1)
display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 0.01$, $\alpha = 1.5$"))
table2 = pd.DataFrame({'Nt': Nt_list, 'error': errs3, 'EOC': eocs3})
display(table2)

display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 3$" ))
table1 = pd.DataFrame({'Nt': Nt_list, 'error': errs13, 'EOC': eocs13})
display(table1)
display(Markdown(r"### 📊 Konvergenstabell for $\kappa = 0.01$, $\alpha = 3$"))
table2 = pd.DataFrame({'Nt': Nt_list, 'error': errs4, 'EOC': eocs4})
display(table2)

### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 0.5$

,Nt,error,EOC
0,100,0.493603,inf
1,200,0.250262,0.979916
2,400,0.126015,0.989837
3,800,0.063231,0.994888
4,1600,0.031672,0.997433
5,3200,0.015850,0.998714


### 📊 Konvergenstabell for $\kappa = 0.01$, $\alpha = 0.5$

,Nt,error,EOC
0,100,0.025655,inf
1,200,0.013286,0.949333
2,400,0.006765,0.973675
3,800,0.003414,0.986575
4,1600,0.001715,0.993220
5,3200,0.000860,0.996593


### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 1.5$

...

,Nt,error,EOC
0,100,0.069291,inf
1,200,0.037423,0.888746
2,400,0.019516,0.939307
3,800,0.009975,0.968211
4,1600,0.005044,0.983719
5,3200,0.002537,0.991759


### 📊 Konvergenstabell for $\kappa = 1$, $\alpha = 3$

,Nt,error,EOC
0,100,1.282962,inf
1,200,0.658626,0.961947
2,400,0.333802,0.980466
3,800,0.168051,0.990097
4,1600,0.084316,0.995015
5,3200,0.042231,0.997499


### 📊 Konvergenstabell for $\kappa = 0.01$, $\alpha = 3$

,Nt,error,EOC
0,100,0.122752,inf
1,200,0.069653,0.817503
2,400,0.037477,0.894183
3,800,0.019500,0.942525
4,1600,0.009955,0.969969
5,3200,0.005031,0.984639


Above I have done a quick study of how the parameters $\kappa$ and $\alpha$ affect the errors and the EOC. We observe that the method obtain an EOC of 1.

### 3-step IMEX runge-kutta method

The 3-step IMEX runge-kutta method is given by:

$
\begin{aligned}
U^{(1)} &= U^n + \tau \left( \mathbf{L} U^{(1)} + \mathbf{N}(U^n) \right), \\
U^{(2)} &= \alpha_{10} U^n + \alpha_{11} U^{(1)} + \beta_1 \tau \left( \mathbf{L} U^{(2)} + \mathbf{N}(U^{(1)}) \right), \\
U^{n+1} &= \alpha_{20} U^n + \alpha_{21} U^{(1)} + \alpha_{22} U^{(2)} + \beta_2 \tau \left( \mathbf{L} U^{n+1} + \mathbf{N}(U^{(2)}) \right).
\end{aligned}
$

In general $U^{n+1}$ is partitioned in an explicit part and an implicit part like this: 


$
\hat{u}^{n+1} = \hat{u}^n - \tau M |\tilde{K}|^2 \left( \widehat{(u^n)^3 - (1+\alpha) \hat{u}^n} \right) + \tau \left( \hat{g}^{n+1} - M \alpha |\tilde{K}|^2 \hat{u}^{n+1} - M \kappa |\tilde{K}|^4 \hat{u}^{n+1} \right)
$ 

#### Step 1:
  

$
\hat{u}^{1} = \hat{u}^n - \tau M |\tilde{K}|^2 \left( \widehat{(u^n)^3 - (1+\alpha) {u}^n} \right) + \tau \left( \hat{g}^{1} - M \alpha |\tilde{K}|^2 \hat{u}^{1} - M \kappa |\tilde{K}|^4 \hat{u}^{1} \right)
$ 

Obtaining:

$
\hat{u}^{1} = \frac{\hat{u}^n + \tau(\hat{g}^1 - M |\tilde{K}|^2 ( \widehat{(u^n)^3 - (1+\alpha) u^n}))}{1 + \tau(\alpha M |\tilde{K}|^2 + M \kappa |\tilde{K}|^4)}
$

#### Step 2:

$
\hat{u}^2 = \alpha_{10} U^n + \alpha_{11} U^{(1)} + \beta_1 \tau(-M |\tilde{K}|^2 ( \widehat{(u^1)^3 - (1+\alpha) {u}^1} +  \hat{g}^{2} - M \alpha |\tilde{K}|^2 \hat{u}^{2} - M \kappa |\tilde{K}|^4 \hat{u}^{2})) 
$

Obtaining: 

$
\hat{u}^2 = \frac{\alpha_{10} \hat{u}^n + \alpha_{11} \hat{u}^{(1)} + \beta_1 \tau (\hat{{g}}^2 - M |\tilde{K}|^2 ( \widehat{(u^1)^3 - (1+\alpha) {u}^1}))}{1 + \beta_1 \tau(\alpha M |\tilde{K}|^2 + M \kappa |\tilde{K}|^4)}
$


#### Step 3:

$
\hat{u}^{n+1} = \alpha_{20} \hat{u}^n + \alpha_{21} \hat{u}^1 + \alpha_{22} \hat{u}^2 + \beta_2 \tau (-M |\tilde{K}|^2 ( \widehat{(u^2)^3 - (1+\alpha) {u}^2} +  \hat{g}^{n+1} - M \alpha |\tilde{K}|^2 \hat{u}^{n+1} - M \kappa |\tilde{K}|^4 \hat{u}^{n+1}))
$

Obtaining:

$
\hat{u}^{n+1} = \frac{\alpha_{20} \hat{u}^n + \alpha_{21} \hat{u}^1 + \alpha_{22} \hat{u}^2 + \beta_2 \tau (\hat{g}^{n+1} - M |\tilde{K}|^2 ( \widehat{(u^2)^3 - (1+\alpha) {u}^2})}{1 + \beta_2 \tau(\alpha M |\tilde{K}|^2 + M \kappa |\tilde{K}|^4)}
$
  
I have used the modified version where:

$\hat{g}^k = \mathbf{G}^{n + 1/2} = \mathbf{G}\left(t_n + \frac{\tau}{2}\right)$

for $k = 1,2, n+1$

In [73]:
def cahn_hilliard_RK(*,kappa,X,Y,U0,t0,T,Nt,alpha,parameterlist,g = None,tau = None):
    x, y = X[0,:], Y[:,0]
    Nx, Ny = len(x), len(y)
    dx, dy = x[1] - x[0], y[1] - y[0]
    alfa10,alfa11,alfa20,alfa21,alfa22,beta1,beta2 = parameterlist

    #Calculating frequencies in frequency domain
    kx = fftfreq(Nx, d=dx/(2*np.pi))
    ky = fftfreq(Ny, d=dy/(2*np.pi))
    KX, KY = np.meshgrid(kx, ky, indexing='ij')

    #Biharmonic operator in frequency domain
    K4 = (KX**2+KY**2)**2

    #Laplacian operator in frequency domain
    K2 = KX**2+KY**2

    #Time step
    t = t0
    dt = (T-t0)/Nt
    U_hat = fft2(U0)
    if tau is not None:
        dt = tau

    #Returing initial conditions
    yield (U_hat, t)
    
    while t < T-dt/2:
        if g is not None:
            G_hat = fft2(g(X,Y,t +dt/2))
        else:
            G_hat= 0
            
        M = 1

        #Copying U_hat as U^n
        U_hat_n = U_hat

        #step1
        U_real_space = ifft2(U_hat_n).real
        nonlinear = U_real_space**3 - (1+alpha)*U_real_space
        nonlinearFourier = fft2(nonlinear)

        U_hat_1 = (U_hat_n + dt*(G_hat - M*K2*nonlinearFourier))/(1+dt*(alpha*M*K2 + kappa*K4))

        #step2
        U_real_space = ifft2(U_hat_1).real
        nonlinear = U_real_space**3 - (1+alpha)*U_real_space
        nonlinearFourier = fft2(nonlinear)

        U_hat_2 = (alfa10*U_hat_n + alfa11*U_hat_1 + beta1*dt*(G_hat - M*K2*nonlinearFourier))/(1+ beta1*dt*(alpha*M*K2+kappa*K4))

        #step3
        U_real_space = ifft2(U_hat_2).real
        nonlinear = U_real_space**3 - (1+alpha)*U_real_space
        nonlinearFourier = fft2(nonlinear)

        U_hat = (alfa20*U_hat_n + alfa21*U_hat_1 + alfa22*U_hat_2 + beta2*dt*(G_hat - M*K2*nonlinearFourier))/(1+beta2*dt*(alpha*M*K2+kappa*K4))

        #Setting mean value
        U_hat[0, 0] = np.mean(ifft2(U_hat)).real * Nx * Ny

        t = t + dt
        yield U_hat, t

In [74]:
#Function calculating the errors and the EOC for different timesteps using the 3-step IMEX
def compute_eoc_cahnHilliard(*,
                          kappa, u_ex, U0, g,
                          X, Y, t0,alpha,parameterlist, T, Nt_list):
    errs_Nt = [] 
    for Nt in Nt_list:
        U_list = cahn_hilliard_RK(kappa=kappa,X=X,Y=Y,U0=U0,t0=t0,T=T,Nt=Nt,alpha=alpha,parameterlist=parameterlist,g = g)
        errs_t = []
        for U, t in U_list:
            U = ifft2(U).real
            U_ex = u_ex(X,Y,t)
            U_err = U - U_ex
            errs_t.append(np.linalg.norm(U_err, np.inf))
        errs_Nt.append(np.array(np.linalg.norm(errs_t, np.inf)))
        
    Nt_list = np.array(Nt_list)
    errs_Nt = np.array(errs_Nt)
    eocs = np.log(errs_Nt[1:]/errs_Nt[:-1])/np.log(Nt_list[:-1]/Nt_list[1:])
    eocs = np.insert(eocs, 0, np.inf)
    return errs_Nt, eocs

In [75]:
#Preparations before computations
Lx,Ly = 16*np.pi,16*np.pi
Nx,Ny =64,64
x = np.linspace(0,Lx,Nx,endpoint=False)
y = np.linspace(0,Ly,Ny,endpoint=False)
X,Y = np.meshgrid(x,y,sparse=True)

#Parameters used
kappa = 0.01
T,t0 = 1,0
alpha = 1.5
M= 1

#Different sets of parameteres for parameter optimazation
parameterlist1 = [3/2,-1/2,0,0,1,1/2,1]
parameterlist2 = [2,-1,1/2,0,1/2,1,1]
parameterlist3 = [2,-1,0,1/2,1/2,1,1/2]
parameterlist4 = [5/2,-3/2,2/3,0,1/3,3/2,1]

Nt_list = [100,200,400,800,1600,3200]


#Analytical solution
u_string_ex = 'sin(x)*cos(y)*exp(-4*kappa*t)'
u_ex1,g_exact1 = manufacture_solution_cahnHilliard(u_string_ex,kappa=0.01)

gk = g_exact1
u = u_ex1
UK = u(X,Y,0)

                    
errs1, eocs1 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=u, U0=UK, g=gk,
                          X=X, Y=Y, t0=t0,alpha=alpha,parameterlist=parameterlist1, T=T, Nt_list=Nt_list)

errs2, eocs2 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=u, U0=UK, g=gk,
                          X=X, Y=Y, t0=t0,alpha=alpha,parameterlist=parameterlist2, T=T, Nt_list=Nt_list)

errs3, eocs3 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=u, U0=UK, g=gk,
                          X=X, Y=Y, t0=t0,alpha=alpha,parameterlist=parameterlist3, T=T, Nt_list=Nt_list)

errs4, eocs4 = compute_eoc_cahnHilliard(
                          kappa=0.01, u_ex=u, U0=UK, g=gk,
                          X=X, Y=Y, t0=t0,alpha=alpha,parameterlist=parameterlist4, T=T, Nt_list=Nt_list)


display(Markdown(r"### 📊 Konvergenstabell for parametersett 1"))
table1 = pd.DataFrame({'Nt': Nt_list, 'error': errs1, 'EOC': eocs1})
display(table1)

display(Markdown(r"### 📊 Konvergenstabell for parametersett 2"))
table2 = pd.DataFrame({'Nt': Nt_list, 'error': errs2, 'EOC': eocs2})
display(table2)

display(Markdown(r"### 📊 Konvergenstabell for parametersett 3"))
table3 = pd.DataFrame({'Nt': Nt_list, 'error': errs3, 'EOC': eocs3})
display(table3)

display(Markdown(r"### 📊 Konvergenstabell for parametersett 4"))
table4 = pd.DataFrame({'Nt': Nt_list, 'error': errs4, 'EOC': eocs4})
display(table4)

### 📊 Konvergenstabell for parametersett 1

,Nt,error,EOC
0,100,0.002774,inf
1,200,0.000711,1.964438
2,400,0.000177,2.004117
3,800,0.000044,2.010911
4,1600,0.000011,2.008287
5,3200,0.000003,2.004676


### 📊 Konvergenstabell for parametersett 2

,Nt,error,EOC
0,100,0.000809,inf
1,200,0.000360,1.168421
2,400,0.000117,1.624302
3,800,0.000033,1.817976
4,1600,0.000009,1.909999
5,3200,0.000002,1.954876


### 📊 Konvergenstabell for parametersett 3

,Nt,error,EOC
0,100,0.002592,inf
1,200,0.000680,1.930449
2,400,0.000173,1.977733
3,800,0.000043,1.993856
4,1600,0.000011,1.998461
5,3200,0.000003,1.999338


### 📊 Konvergenstabell for parametersett 4

,Nt,error,EOC
0,100,0.004196,inf
1,200,0.001398,1.585256
2,400,0.000406,1.783810
3,800,0.000110,1.889929
4,1600,0.000028,1.944551
5,3200,0.000007,1.972087


Above I quick study performance with different sets of parameters. We can see that parameterset 1 gives the best EOC. I will use this set of parameteres for my simulation.

In [76]:
#Preparation before computation
Lx,Ly = 1/2,1/2
Nx,Ny =256,256
x = np.linspace(0,Lx,Nx,endpoint=False)
y = np.linspace(0,Ly,Ny,endpoint=False)
X,Y = np.meshgrid(x,y)

#Parameters used
kappa = 0.0025**2
T1,T2 = 4,0.01
t0 = 0
tau1,tau2 = 10**(-3), 10**(-4)
alpha = 1
M= 1
Nt = 405

#Creating two random sets of initial conditions:

#random u01:
rng = np.random.default_rng(12345)
noise = 0.05
u0_base = 0 
U0 = np.ones_like((Ny, Nx))
U01= np.full((Ny, Nx), u0_base) +  noise*rng.standard_normal((Ny, Nx))

#random u02:
rng = np.random.default_rng(12345)
noise = 0.05
u0_base = -0.45
U0 = np.ones_like((Ny, Nx))
U02 = np.full((Ny, Nx), u0_base) +  noise*rng.standard_normal((Ny, Nx))

parameterlist1 = [3/2,-1/2,0,0,1,1/2,1]

In [80]:
#Final simulation of the Cahn-Hilliard equation using the 3-step IMEX RK solver.
Td = 5
n = 40000
tau = Td/n

RK_cahn_hilliard1 = cahn_hilliard_RK(kappa=kappa,X=X,Y=Y,U0=U01,t0=t0,T=Td,Nt=Nt,alpha=alpha,parameterlist=parameterlist1,g = None,tau = tau)
RK_cahn_hilliard2 = cahn_hilliard_RK(kappa=kappa,X=X,Y=Y,U0=U02,t0=t0,T=Td,Nt=Nt,alpha=alpha,parameterlist=parameterlist1,g = None,tau = tau)

ctr = 0
RKCH1_real = []
RKCH1_time = []
RKCH2_real = []
RKCH2_time = []
ctr_list = [0,1,4,6,10,20,100,500,1000,5000,10000,20000,39999]


for u,t in RK_cahn_hilliard1:
    if ctr in ctr_list:
        real_vals = ifft2(u).real
        RKCH1_real.append(real_vals)
        RKCH1_time.append(t)
    ctr += 1

ctr = 0

for u,t in RK_cahn_hilliard2:
    if ctr in ctr_list:
        real_vals = ifft2(u).real
        RKCH2_real.append(real_vals)
        RKCH2_time.append(t)
    ctr += 1

In [81]:
# RKCH1 animation
fig1, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(X, Y, RKCH1_real[0], levels=100, cmap='magma', vmin=-1, vmax=1)
cbar = fig1.colorbar(contour, ax=ax)
ax.set_title(r"Simulation with $u_0 = -0.45 + 0.05rand(x,y)$")
ax.grid(False)
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
frame_text = ax.text(0.02, 0.95, "", transform=ax.transAxes, color='white')

def update1(i):
    for artist in ax.collections:
        artist.remove()
    contour = ax.contourf(X, Y, RKCH1_real[i], levels=100, cmap='magma', vmin=-1, vmax=1)
    frame_text.set_text(f"Time: {np.round(RKCH1_time[i],4)} s")
    return list(ax.collections) + [frame_text]

ani1 = FuncAnimation(fig1, update1, frames=len(RKCH1_real), interval=30, blit=False)
ani1.save('CahnHilliard_RK_u1.gif', writer=PillowWriter(fps=1))
plt.close(fig1)

# RKCH2 animation
fig2, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(X, Y, RKCH2_real[0], levels=100, cmap='magma', vmin=-1, vmax=1)
cbar = fig2.colorbar(contour, ax=ax)
ax.set_title(r"Simulation with $u_0 = 0 + 0.05rand(x,y)$")
ax.grid(False)
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
frame_text = ax.text(0.02, 0.95, "", transform=ax.transAxes, color='white')

def update2(i):
    for artist in ax.collections:
        artist.remove()
    contour = ax.contourf(X, Y, RKCH2_real[i], levels=100, cmap='magma', vmin=-1, vmax=1)
    frame_text.set_text(f"Time: {np.round(RKCH2_time[i],4)} s")
    return list(ax.collections) + [frame_text]

ani2 = FuncAnimation(fig2, update2, frames=len(RKCH2_real), interval=30, blit=False)
ani2.save('CahnHilliard_RK_u2.gif', writer=PillowWriter(fps=1))
plt.close(fig2)